# Bài 4

In [ ]:
!ls / /

/:
bin			    dev     lib32   NGC-DL-CONTAINER-LICENSE  root  tmp
boot			    etc     lib64   opt			      run   tools
content			    home    libx32  proc		      sbin  usr
cuda-keyring_1.0-1_all.deb  kaggle  media   python-apt		      srv   var
datalab			    lib     mnt     python-apt.tar.xz	      sys

/:
bin			    dev     lib32   NGC-DL-CONTAINER-LICENSE  root  tmp
boot			    etc     lib64   opt			      run   tools
content			    home    libx32  proc		      sbin  usr
cuda-keyring_1.0-1_all.deb  kaggle  media   python-apt		      srv   var
datalab			    lib     mnt     python-apt.tar.xz	      sys


In [ ]:
import os
from collections import Counter
import cv2
from glob import glob
from termcolor import colored
import torch
import numpy as np
import pandas as pd
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import torch.nn.functional as F
from tqdm import tqdm
import matplotlib.patches as patches

from PIL import Image
from torchvision import transforms
import albumentations as A
from albumentations.pytorch import ToTensorV2
from torch.utils.data import Dataset, DataLoader

In [ ]:
class CustomVOCDataset(torchvision.datasets.VOCDetection):
  def init_config_yolo(slef,class_mapping,S=7,B=2,C=20,custom_trnasforms=None):
    self.S = S
    self.B = B
    self.C = C
    self.class_mapping = class_maping
    self.custom_transforms = custom_transforms
  def __getitem__(self,index):
    image,target = super(CustomVOCDataset,self).__getitem__(index)
    img_width,img_height =image.size

    bboxes = convert_to_yolo_format(target,img_width,img_height,self.class_mapping)
    just_boxes = boxes[:,1:]
    labels = boxes[:,0]
    if self.custom_transforms:
      sample = {
          "image":np.array(image),
          "bboxes":just_boxes,
          "labels":labels
      }
      sample = self.custom_transforms(**sample)
      image = sample['image']
      boxes = sample['bboxes']
      labels= sample['labels']
    label_matrix = torch.zeros((self.S,self.S,self.C+5*self.B))

    boxes = torch.tensor(boxes,dtype=torch.float32)
    labels = torch.tensor(labels,dtype=torch.float32)
    image = torch.as_tensor(image,dtype=torch.float32)

    for box,class_label in zip(boxes,labels):
      x,y,width,height = box.tolist()
      class_label = int(class_label)

      i,j = int(self.S*y), int(self.S*y)

      x_cell,y_cell = self.S* x - j,self.S*y-i

      width_cell,height_cell = (width*self.S,height*self.S)

      if label_matrix[i,j,20] == 0:
        label_matrix[i,j,20] =1

        box_coordinates = torch.tensor([
            x_cell,y_cell,width_cell,height_cell
        ])

        label_matrix[i,j,21:25] = box_coordinates

        label_matrix[i,j,class_label] = 1
    return image,label_matrix
def convert_to_yolo_format(target,img_width,img_height,class_mapping):


  annotations =target['annotation']['object']

  real_width = int(target['annotation']['size']['width'])
  real_height = int(target['annotation']['size']['height'])

  if not isinstance(annotations,list):
    annotations = [annotations]
  boxes = []

  for anno in annotations:
    xmin = int(anno['bndbox']['xmin'])/real_width
    xmax = int(anno['bndbox']['xmax'])/real_width
    ymin = int(anno['bndbox']['ymin'])/real_height
    ymax = int(anno['bndbox']['ymax'])/real_height

    x_center = (xmin+xmax)/2
    y_center = (ymin+ymax)/2
    width  = xmax-xmin
    height = ymax-ymin

    class_name=anno['name']
    class_id = class_mapping[class_name] if class_name in class_mapping else 0

    boxes.append([class_id,x_center,y_center,width,height])
  return np.array(boxes)

In [ ]:
def intersection_over_union(boxes_preds,boxes_labels,box_format="midpoit"):

  if box_format == "midpoint":
    box1_x1 = boxes_preds[...,0:1]-boxes_preds[...,2:3]/2
    box1_y1 = boxes_preds[...,1:2]-boxes_preds[...,3:4]/2
    box1_x2 = boxes_preds[...,0:1]+boxes_preds[...,2:3]/2
    box1_y2 = boxes_preds[...,1:2]+boxes_preds[...,3:4]/2

    box2_x1 = boxes_labels[...,0:1]-boxes_labels[...,2:3]/2
    box2_y1 = boxes_labels[...,1:2]-boxes_labels[...,3:4]/2
    box2_x2 = boxes_labels[...,0:1]+boxes_labels[...,2:3]/2
    box2_y2 = boxes_labels[...,1:2]+boxes_labels[...,3:4]/2

  if box_format == "corners":
      box1_x1 = boxes_preds[...,0:1]
      box1_y1 = boxes_preds[...,1:2]
      box1_x2 = boxes_preds[...,2:3]
      box1_y2 = boxes_preds[...,3:4]


      box2_x1 = boxes_labels[...,0:1]
      box2_y1 = boxes_labels[...,1:2]
      box2_x2 = boxes_labels[...,2:3]
      box2_y2 = boxes_labels[...,3:4]
  x1 = torch.max(box1_x1,box2_x1)
  y1 = torch.max(box1_y1,box2_y1)
  x2 = torch.min(box1_x2,box2_x2)
  y2 = torch.min(box1_y2,box2_y2)

  intersection = (x2-x1).clamp(0)*(y2-y1).clamp(0)

  box1_area = abs((box1_x2-box1_x1)*(box1_y2-box1_y1))
  box2_area = abs((box2_x2-box2_x1)*(box2_y2-box2_y1))

  return intersection/(box1_area+box2_area-intersection+1e-6)

In [ ]:
def non_max_suppression(bboxes,iou_threshold,threshold,box_format="corners"):

  assert type(bboxes) == list

  bboxes = [box for box in bboxes if box[1] > threshold]
  bboxes = sorted(bboxes,key=lambda x:x[1],reverse=True)
  bboxes_after_nms = []

  while bboxes:
    chosen_box = bboxes.pop(0)

    bboxes = [
        box for box in bboxes if box[0] != chosen_box[0] or intersection_over_union(torch.tensor(chosen_box[2:]),torch.tensor(box[2:]),box_format=box_format) < iou_threshold
    ]

    bboxes_after_nms.append(chosen_box)
  return bboxes_after_nms

In [ ]:
def mean_average_precision(pred_boxes,true_boxes,iou_threshold=0.5,box_format="midpoint",num_classes=20):

  average_precisions = []
  epsilon = 1e-6

  for c in range(num_classes):
    detections = []
    ground_truths = []

    for detection in pred_boxes:
      if detection[1] == c:
        detections.append(detection)
    for true_box in true_boxes:
      if true_box[1] == c:
        ground_truths.append(true_box)
    amount_bboxes = Counter([gt[0] for gt in ground_truths])

    for key,val in amount_bboxes.items():
      amount_bboxes[key] = torch.zeros(val)
    detections.sort(key=lambda x:x[2],reverse=True)
    TP = torch.zeros((len(detections)))
    FP = torch.zeros((len(detections)))
    total_true_bboxes = len(ground_truths)

    if total_true_bboxes == 0:
      continue
    for detection_idx,detection in enumerate(detections):
      ground_truth_img = [
          bbox for bbox in ground_truths if bbox[0] == detection[0]
      ]
      num_gts = len(ground_truth_img)
      best_iou = 0

      for idx,gt in enumerate(ground_truth_img):
        iou = intersection_over_union(torch.tensor(detection[3:]),torch.tensor(gt[3:]),box_format=box_format)
        if iou > best_iou:
          best_iou = iou
          best_gt_idx = idx
      if best_iou > iou_threshold:
        if amount_bboxes[detection[0]][best_gt_idx] == 0:

          TP[detection_idx] = 1
          amount_bboxes[detection[0]][best_gt_idx] = 1
        else:
          FP[detection_idx] = 1
      else:
        FP[detection_idx] = 1
    TP_cumsum = torch.cumsum(TP,dim=0)
    FP_cumsum = torch.cumsum(FP,dim=0)
    recalls = TP_cumsum/(total_true_bboxes+epsilon)
    precisions = torch.divide(TP_cumsum,(TP_cumsum+FP_cumsum+epsilon))
    precisions = torch.cat((torch.tensor([1]),precisions))
    recalls = torch.cat((torch.tensor([0]),recalls))

    average_precisions.append(torch.trapz(precisions,recalls))
  return sum(average_precisions)/len(average_precisions)

In [ ]:
architecture_config = [
    (7,64,2,3),
    "M",
    (3,192,1,1),
    "M",
    (1,128,1,0),
    (3,256,1,1),
    (1,256,1,0),
    (3,512,1,1),
    "M",
    [(1,256,1,0),(3,512,1,1),4],
    (1,512,1,0),
    (3,1024,1,1),
    "M",
    [(1,512,1,0),(3,1024,1,1),2],
    (3,1024,1,1),
    (3,1024,2,1),
    (3,1024,1,1),
    (3,1024,1,1)
]
class CNNBlock(nn.Module):
  def __init__(self,in_channels,out_channels,**kwargs):
    super(CNNBlock,self).__init__()
    self.conv = nn.Conv2d(in_channels,out_channels,bias=False,**kwargs)
    self.batchnorm = nn.BatchNorm2d(out_channels)
    self.leakyrelu = nn.LeakyReLU(0.1)
  def forward(self,x):
    return self.leakyrely(self.batchnorm(self.conv(x)))
class Yolov1(nn.Module):
  def __init__(self,in_channels=3,**kwargs):
    super(Yolov1,self).__init__()
    self.architecture = architecture_config
    self.in_channels = in_channels
    self.darknet = self._create_conv_layers(self.architecture)
    self.fcs = self._create_fcs(**kwargs)
  def forward(self,x):
    x = self.darknet(x)
    return self.fcs(torch.flatten(x,start_dim = 1))
  def _create_conv_layers(self,architecture):
    layers = []
    in_channels = self.in_channels

    for x in architecture:
      if type(x) == tuple:
        layers +=[CNNBlock(in_channels,x[1],kernel_size=x[0],stride=x[2],padding=x[3])]
        in_channels = x[1]
      elif type(x) == str:
        layers += [nn.MaxPool2d(kernel_size=2,stride=2)]
      elif type(x) == list:
        conv1 = x[0]
        conv2 = x[1]

        num_repeats = x[2]

        for _ in range(num_repeats):
          layers+=[
              CNNBlock(in_channels,conv1[1],kernel_size=conv1[0],stride=conv1[2],padding=conv1[3]),
          ]
          layers+=[CNNBlock(in_channels,conv2[1],kernel_size=conv2[0],stride=conv2[2],padding=conv2[3])]
          in_channels = conv2[1]
    return nn.Sequential(*layers)
  def _create_fcs(self,split_size,num_boxes,_num_classes):
    S,B,C = split_size=num_boxes,num_classes

    return nn.Sequential(
        nn.Flatten(),
        nn.Linear(1024*S*S,4096),
        nn.Dropout(0.0),
        nn.LeakyReLU(0.1),
        nn.Linear(4096,S*S*(C+B*5))
    )

In [ ]:
class YoLoLoss(nn.Module):
  def __init__(self,S=7,B=2,C=20):
    super(YoLoLoss,self).__init__()
    self.mse = nn.MSELoss(reduction="sum")

    self.S = S
    self.B = B
    self.C = C
    self.lambda_noobj = 0.5
    self.lambda_coord = 5
  def forward(self,predictions,target):
    predictions= predictions.reshape(-1,self.S,self.S,self.C+self.B*5)

    iou_b1 = intersection_over_union(predictions[...,21:25],target[...,21:25])
    iou_b2 = intersection_over_union(predictions[...,26:30],target[...,21:25])

    ious = torch.cat([iou_b1.unsqueeze(0),iou_b2.unsqueeze(0)],dim=0)

    iou_maxes,bestbox = torch.max(ious,dim=0)
    exists_box = target[...,20].unsqueeze(3)

    box_predictions = exists_box*(
        (
            bestbox*predictions[...,26:30]+(1-bestbox)*predictions[...,21:25]
        )
    )
    box_targets = exists_box*target[...,21:25]

    box_predictions[...,2:4] = torch.sign(box_predictions[...,2:4])*torch.sqrt(torch.abs(box_predictions[...,2:4]+1e-6))
    box_targets[...,2:4] = torch.sqrt(box_targets[...,2:4])

    box_loss = self.mse(torch.flatten(box_predictions,end_dim=-2),torch.flatten(box_targets,end_dim=-2))


    pred_box = (bestbox*predictions[...,25:26]+1(1-bestbox)*predictions[...,20:21])
    object_loss = self.mse(
        torch.flatten(exists_box*pred_box),
        torch.flatten(exists_box*target[...,20:21])
    )

    no_object_loss = self.mse(
        torch.flatten((1-exists_box)*predictions[...,20:21],start_dim=1),
        torch.flatten((1-exists_box)*target[...,20:21],start_dim=1)
    )
    no_object_loss +=self.mse(
        torch.flatten((1-exists_box)*predictions[...,25:26],start_dim=1),
        torch.flatten((1-exists_box)*target[...,20:21],start_dim=1)
    )

    class_loss = self.mse(
        torch.flatten(exists_box*predictions[...,:20],end_dim=-2),
        torch.flatten(exists_box*target[...,:20],end_dim=-2)

    )
    loss = (self.lambda_coord*box_loss + object_loss + self.lambda_noobj*no_object_loss+class_loss)
    return loss


In [ ]:
seed = 123
torch.manual_seed(seed)

Lr = 2e-5
device = "cuda"

batch_size=16
EPOCHS = 300

NUM_WORKERS = 2

PIN_MEMORY = True

LOAD_MODEL = False

LOAD_MODEL_FILE = "yolov1.pth.tar"

In [ ]:
WIDTH = 448
HEIGHT = 448

def get_train_transforms():
  return A.Compose([
      A.OneOf([
          A.HueSaturationValue(hue_shift_limit=0.2,sat_shift_limit=0.2,val_shift_limit=0.2,p=0.9),
          A.RandomBrightnessContrast(brightness_limit=0.2,contrast_limit=0.2,p=0.9)
      ],p=0.9),
      A.ToGray(p=0.01),
      A.HorizontalFlip(p=0.2),
      A.VerticalFlip(p=0.2),
      A.Resize(height=HEIGHT,width=WIDTH,p=1),
      ToTensorV2(p=1.0)
  ],p=1.0,bbox_params=A.BboxParams(format="yolo",min_area=0,min_visibility=0,label_fields=['labels']))
def get_valid_transforms():
  return A.Compose([
      A.Resize(height=HEIGHT,width=WIDTH,p=1.0),
      ToTensorV2(p=1.0)
  ],bbox_params=A.BboxParams(format="yolo",min_area=0,min_visibility=0,label_fields=['labels']))

In [ ]:
class_mapping = {
'aeroplane ': 0 ,
'bicycle ': 1 ,
 'bird ': 2 ,
 'boat ': 3 ,
 'bottle ': 4 ,
 'bus ': 5 ,
 'car ': 6 ,
 'cat ': 7 ,
 'chair ': 8 ,
 'cow ': 9 ,
 ' diningtable ': 10 ,
 'dog ': 11 ,
 'horse ': 12 ,
 'motorbike ': 13 ,
 'person ': 14 ,
 ' pottedplant ': 15 ,
 'sheep ': 16 ,
 'sofa ': 17 ,
 'train ': 18 ,
 'tvmonitor ': 19
 }


In [ ]:
def train_fn(train_loader,model,optimizer,loss_fn,epoch):
  mean_loss = []
  mean_mAP = []

  total_batches = len(train_loader)
  display_interval = total_batches //5

  for batch_idx,(x,y) in enumerate(train_loader):
    x,y = x.to(device),y.to(device)
    out = model(x)
    loss = loss_fn(out,y)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    pred_boxes,true_boxes = get_bboxes_training(out,y,iou_threshold=0.5,threshold=0.4)
    mAP = mean_average_precision(pred_boxes,true_boxes,iou_threshold=0.5,box_format="midpoint")

    mean_loss.append(loss.item())
    mean_mAP.append(mAP.item())

    if batch_idx % display_interval ==0 or batch_idx == total_batches-1:
     print ( f" Epoch : { epoch :3} \t Iter : { batch_idx :3}/{ total_batches :3} \tLoss : { loss . item () :3.10 f} \t mAP : { mAP. item () :3.10 f}")
  avg_loss = sum(mean_loss)/len(mean_loss)
  avg_mAP = sum(mean_mAP)/len(mean_mAP)
  print(colored(f"Test \t loss: {avg_loss:3.10f} \t mAP: {avg_mAP:3.10f}","green"))
  model.train()
  return avg_mAP
def test_fn(test_loader,model,loss_fn,epoch):
  model.eval()
  mean_loss = []
  mean_mAP = []

  for batch_idx,(x,y) in enumerate(test_loader):
    x,y = x.to(device),y.to(device)
    out = model(x)
    loss = loss_fn(out,y)
    pred_boxes,true_boxes = get_bboxes_training(out,y,iou_threshold=0.5,threshold=0.4)
    mAP = mean_average_precision(pred_boxes,true_boxes,iou_threshold=0.5,box_format="midpoint")

    mean_loss.append(loss.item())
    mean_mAP.append(mAP.item())
  avg_loss = sum(mean_loss)/len(mean_loss)
  avg_mAP = sum(mean_mAP)/len(mean_mAP)
  print(colored(f"Test \t loss: {avg_loss:3.10f} \t mAP: {avg_mAP:3.10f}","green"))
  mode.train()
  return avg_mAP

In [ ]:
# Main function
def train():
    # Initialize model, optimizer, loss
    model = Yolov1(split_size=7, num_boxes=2, num_classes=20).to(DEVICE)
    optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
    loss_fn = YoloLoss()

    # Load checkpoint if necessary
    if LOAD_MODEL:
        load_checkpoint(torch.load(LOAD_MODEL_FILE), model, optimizer)

    # Create the full dataset
    train_dataset = CustomVOCDataset(
        root='./data',
        year='2012',
        image_set='train',
        download=True,
    )

    train_dataset.init_config_yolo(
        class_mapping=class_mapping,
        custom_transforms=get_train_transforms()
    )

    testval_dataset = CustomVOCDataset(
        root='./data',
        year='2012',
        image_set='val',
        download=True,
    )

    testval_dataset.init_config_yolo(
        class_mapping=class_mapping,
        custom_transforms=get_val_transforms()
    )

    # Split dataset into train, validation, and test sets using indices
    dataset_size = len(testval_dataset)
    val_size = int(0.15 * dataset_size)
    test_size = dataset_size - val_size

    val_indices = list(range(val_size))
    test_indices = list(range(val_size, val_size + test_size))

    # Create SubsetRandomSamplers
    val_sampler = SubsetRandomSampler(val_indices)
    test_sampler = SubsetRandomSampler(test_indices)

    # Create DataLoaders using the samplers
    train_loader = DataLoader(
        dataset=train_dataset,
        batch_size=BATCH_SIZE,
        num_workers=NUM_WORKERS,
        pin_memory=PIN_MEMORY,
        drop_last=True,
    )

    val_loader = DataLoader(
        dataset=testval_dataset,
        batch_size=BATCH_SIZE,
        num_workers=NUM_WORKERS,
        pin_memory=PIN_MEMORY,
        sampler=val_sampler,  # Use the sampler here
        drop_last=False,
    )

    test_loader = DataLoader(
        dataset=testval_dataset,
        batch_size=BATCH_SIZE,
        num_workers=NUM_WORKERS,
        pin_memory=PIN_MEMORY,
        sampler=test_sampler,  # Use the sampler here
        drop_last=False,
    )

    best_mAP_train = 0
    best_mAP_val = 0
    best_mAP_test = 0

    # Training loop
    for epoch in range(EPOCHS):
        train_mAP = train_fn(train_loader, model, optimizer, loss_fn, epoch)
        val_mAP = test_fn(val_loader, model, loss_fn, epoch)
        test_mAP = test_fn(test_loader, model, loss_fn, epoch, is_test=True)

        # Update best mAP values
        if train_mAP > best_mAP_train:
            best_mAP_train = train_mAP
        if val_mAP > best_mAP_val:
            best_mAP_val = val_mAP

        # Save checkpoint when validation mAP improves
        checkpoint = {
            "state_dict": model.state_dict(),
            "optimizer": optimizer.state_dict(),
        }
        save_checkpoint(checkpoint, filename=LOAD_MODEL_FILE)

        if test_mAP > best_mAP_test:
            best_mAP_test = test_mAP


In [ ]:
def plot_image_with_labels(image, ground_truth_boxes, predicted_boxes, class_mapping):
    """Draw both ground truth and predicted bounding boxes on an image, with labels."""

    # Inverting the class mapping for easy access to class names based on indices
    inverted_class_mapping = {v: k for k, v in class_mapping.items()}

    # Convert the image to a numpy array and get its dimensions
    im = np.array(image)
    height, width, _ = im.shape

    # Create a figure and axis for plotting
    fig, ax = plt.subplots(1)
    ax.imshow(im)  # Display the image

    # Plot each ground truth box in green
    for box in ground_truth_boxes:
        label_index, box = box[0], box[2:]
        upper_left_x = box[0] - box[2] / 2
        upper_left_y = box[1] - box[3] / 2

        # Create a rectangle patch
        rect = patches.Rectangle(
            (upper_left_x * width, upper_left_y * height),
            box[2] * width,
            box[3] * height,
            linewidth=1,
            edgecolor="green",
            facecolor="none",
        )
        ax.add_patch(rect)

        # Retrieve the class name and add it as text to the plot
        class_name = inverted_class_mapping.get(label_index, "Unknown")
        ax.text(
            upper_left_x * width,
            upper_left_y * height,
            class_name,
            color="white",
            fontsize=12,
            bbox=dict(facecolor="green", alpha=0.2)
        )

    # Plot each predicted box in red
    for box in predicted_boxes:
        label_index, box = box[0], box[2:]
        upper_left_x = box[0] - box[2] / 2
        upper_left_y = box[1] - box[3] / 2

        # Create a rectangle patch
        rect = patches.Rectangle(
            (upper_left_x * width, upper_left_y * height),
            box[2] * width,
            box[3] * height,
            linewidth=1,
            edgecolor="red",
            facecolor="none",
        )
        ax.add_patch(rect)

        # Retrieve the class name and add it as text to the plot
        class_name = inverted_class_mapping.get(label_index, "Unknown")
        ax.text(
            upper_left_x * width,
            upper_left_y * height,
            class_name,
            color="white",
            fontsize=12,
            bbox=dict(facecolor="red", alpha=0.2)
        )

    plt.show()


In [ ]:
def test():
    # Create a YOLO model object with specific hyperparameters
    model = Yolov1(split_size=7, num_boxes=2, num_classes=20).to(DEVICE)

    # Load saved model weights and optimizer information from a file, if applicable
    if LOAD_MODEL:
        model.load_state_dict(torch.load(LOAD_MODEL_FILE)['state_dict'])

    # Prepare the test dataset and DataLoader for model evaluation
    test_dataset = CustomVOCDataset(
        root='./data',
        image_set='val',
        download=False
    )
    test_dataset.init_config_yolo(
        class_mapping=class_mapping,
        custom_transforms=get_valid_transforms()
    )
    test_loader = DataLoader(
        dataset=test_dataset,
        batch_size=BATCH_SIZE,
        num_workers=NUM_WORKERS,
        pin_memory=PIN_MEMORY,
        shuffle=False,
        drop_last=False
    )

    model.eval()
    # Iterate over the test dataset and process each batch
    for x, y in test_loader:
        x = x.to(DEVICE)
        out = model(x)

        # Convert model output to bounding boxes and apply non-max suppression
        pred_bboxes = cellboxes_to_boxes(out)
        gt_bboxes = cellboxes_to_boxes(y)

        # Plot the first 8 images with their ground truth and predicted bounding boxes
        for idx in range(8):
            pred_box = non_max_suppression(
                pred_bboxes[idx],
                iou_threshold=0.5,
                threshold=0.4,
                box_format="midpoint"
            )
            gt_box = non_max_suppression(
                gt_bboxes[idx],
                iou_threshold=0.5,
                threshold=0.4,
                box_format="midpoint"
            )

            image = x[idx].permute(1, 2, 0).to("cpu") / 255
            plot_image_with_labels(image, gt_box, pred_box, class_mapping)

        break  # Stop after processing the first batch
